此处用于聚合所有模块进行训练

In [ ]:
# master_experiment_runner_v2.py
from ultralytics import YOLO
import torch
from dbl_loss_tsd import DBLDetectionTrainer # 确保dbl_loss_tsd.py和这个脚本在同一目录

def run_experiment(model_yaml: str, experiment_name: str, use_dbl_loss: bool):
    """
    A standardized function to run a full experiment, now with adaptive training strategies.
    """
    print(f"\n{'='*25} STARTING EXPERIMENT: {experiment_name} {'='*25}")
    
    # === Step 1: Build model and transfer weights ===
    print(f"--- Building model from '{model_yaml}' and loading 'yolo11n.pt' weights ---")
    # For experiments with new modules, this loads pretrained weights into the matching backbone parts.
    model = YOLO(model_yaml).load("yolo11n.pt")
    
    # === Step 2: Setup Trainer and Callbacks ===
    trainer_class = DBLDetectionTrainer if use_dbl_loss else None
    setup_tensorboard(experiment_name)
    try:
        model.add_callback("on_fit_epoch_end", tb_on_fit_epoch_end)
        model.add_callback("on_train_end", tb_on_train_end)
    except NameError:
        print("TensorBoard callbacks not found.")
    _device = 0 if torch.cuda.is_available() else 'cpu'

    # <<< MODIFICATION: Adaptive Training Strategy Selection >>>
    if use_dbl_loss:
        # --- STRATEGY B: Global Adaptive Fine-tuning (for DBL Loss) ---
        print("\n--- Applying STRATEGY B: Global Adaptive Fine-tuning ---")
        print("Reason: DBL_TSD loss is a fundamental change. All layers will train together.")
        
        model.train(
            trainer=trainer_class,
            data="pest24.yaml",
            seed=42,
            epochs=80, # Can use a slightly shorter total epoch count for this aggressive strategy
            batch=24,
            workers=2,
            lr0=5e-5,      # A moderate initial learning rate
            lrf=1e-6,
            optimizer="AdamW",
            weight_decay=0.001,
            patience=20,
            name=f"{experiment_name}_global_adapt",
            device=_device,
            cache=True,
            amp=True
        )
    else:
        # --- STRATEGY A: Two-Stage Fine-tuning (for Architectural Changes) ---
        print("\n--- Applying STRATEGY A: Two-Stage Fine-tuning ---")
        print("Reason: New architectural modules need to be warmed up.")

        # STAGE 1: Warmup New/Modified Modules
        print("\n--- STAGE 1: WARMUP - Training only the Head/Neck ---")
        for i in range(11): # Freeze backbone
            if i < len(model.model.model):
                for param in model.model.model[i].parameters():
                    param.requires_grad = False
        
        model.train(
            trainer=trainer_class, data="pest24.yaml", epochs=30, lr0=1e-4, 
            batch=24, workers=2, device=_device, name=f"{experiment_name}_stage1_warmup"
        )

        # STAGE 2: Global Fine-tuning
        print("\n--- STAGE 2: FINETUNE - Training all layers ---")
        for param in model.model.parameters():
            param.requires_grad = True
            
        model.train(
            trainer=trainer_class, data="pest24.yaml", epochs=70, lr0=5e-6, lrf=1e-6,
            batch=24, workers=2, device=_device, name=f"{experiment_name}_stage2_global",
            seed=42, optimizer="AdamW", weight_decay=0.001, patience=20
        )

    # === Final Step: Save final model ===
    final_weights_path = f"weights/{experiment_name}_final.pt"
    model.save(final_weights_path)
    print(f"\n{'='*25} EXPERIMENT {experiment_name} COMPLETE {'='*25}")
    print(f"Final model saved to {final_weights_path}")

In [ ]:

# --- 单独验证 ---
# 实验1: 仅Fusion Neck (架构改动 -> 策略A)
run_experiment("yamls/yolo11n-fusion.yaml", "exp_fusion_only", use_dbl_loss=False)

In [ ]:
# 实验2: ECA + Fusion (架构改动 -> 策略A)
run_experiment("yamls/yolo11n-eca-fusion.yaml", "exp_eca_fusion", use_dbl_loss=False)

In [ ]:
# 实验3: CSAB + Fusion (架构改动 -> 策略A)
run_experiment("yamls/yolo11n-csab-fusion.yaml", "exp_csab_fusion", use_dbl_loss=False)

In [ ]:
# 实验4: DBL + Fusion (架构+损失改动 -> 策略B)
# The loss change is more fundamental, so we choose the global strategy.
run_experiment("yamls/yolo11n-fusion.yaml", "exp_dbl_fusion", use_dbl_loss=True)

In [ ]:
    
# 实验5: ECA + CSAB + Fusion (架构改动 -> 策略A)
run_experiment("yamls/yolo11n-ultimate.yaml", "exp_ultimate_no_dbl", use_dbl_loss=False)

In [ ]:
# 实验6: ECA + CSAB + Fusion + DBL (架构+损失改动 -> 策略B)
# Again, the fundamental loss change dictates the strategy.
run_experiment("yamls/yolo11n-ultimate.yaml", "exp_ultimate_with_dbl", use_dbl_loss=True)